Install Dependencies and Import Libraries

In [6]:
# Install necessary packages
!pip install sqlalchemy pandas

# Import required modules
from sqlalchemy import create_engine, Column, Integer, String, Boolean, ForeignKey, Date, Float
from sqlalchemy.orm import relationship, sessionmaker, declarative_base
import datetime
import random
import pandas as pd
from google.colab import files

# Set up the base declarative class
Base = declarative_base()


Define ORM Classes

In [7]:
# BehaviorRecord ORM Class
class BehaviorRecord(Base):
    __tablename__ = 'behavior_records'
    id = Column(Integer, primary_key=True)
    inmate_id = Column(Integer, ForeignKey('inmates.id', ondelete='CASCADE'))
    date = Column(Date)
    description = Column(String)
    behavior_type = Column(String)

# Parole ORM Class
class Parole(Base):
    __tablename__ = 'paroles'
    id = Column(Integer, primary_key=True)
    inmate_id = Column(Integer, ForeignKey('inmates.id', ondelete='CASCADE'), unique=True)
    parole_date = Column(Date)
    eligibility_status = Column(Boolean)

    # Relationship back to the inmate (one-to-one)
    inmate = relationship("Inmate", back_populates="parole_status", uselist=False)

# MedicalRecord ORM Class
class MedicalRecord(Base):
    __tablename__ = 'medical_records'
    id = Column(Integer, primary_key=True)
    inmate_id = Column(Integer, ForeignKey('inmates.id', ondelete='CASCADE'), unique=True)
    medical_condition = Column(String)
    treatment_plan = Column(String)

    # Relationship back to the inmate (one-to-one)
    inmate = relationship("Inmate", back_populates="medical_records", uselist=False)

# Cell ORM Class
class Cell(Base):
    __tablename__ = 'cells'
    id = Column(Integer, primary_key=True)
    cell_number = Column(Integer, unique=True)
    capacity = Column(Integer)
    current_inmates = relationship("Inmate", back_populates="cell")

# Inmate ORM Class
class Inmate(Base):
    __tablename__ = 'inmates'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    age = Column(Integer)
    crime = Column(String)
    sentence_duration = Column(Integer)  # Number of years sentenced
    time_served = Column(Float, default=0)  # Track time served
    cell_id = Column(Integer, ForeignKey('cells.id'))

    # Relationships
    behavior_records = relationship("BehaviorRecord", backref="inmate")
    parole_status = relationship("Parole", uselist=False, back_populates="inmate", foreign_keys=[Parole.inmate_id])
    medical_records = relationship("MedicalRecord", uselist=False, back_populates="inmate", foreign_keys=[MedicalRecord.inmate_id])
    cell = relationship("Cell", back_populates="current_inmates")


Additional Helper Classes (Warden, Guard, WorkAssignment)

In [8]:
# Warden Class
class Warden:
    def __init__(self, name, staff_id):
        self.name = name
        self.staff_id = staff_id

    def assign_cell(self, inmate, cell):
        return cell.add_inmate(inmate)

    def approve_parole(self, inmate):
        if inmate.get_parole_eligibility():
            inmate.parole_status.eligibility_status = True
            return f"Inmate {inmate.name}'s parole has been approved"
        else:
            return f"Inmate {inmate.name} is not eligible for parole"

# Guard Class
class Guard:
    def __init__(self, name, staff_id):
        self.name = name
        self.staff_id = staff_id

    def monitor_inmate(self, inmate):
        return f"Guard {self.name} is monitoring inmate {inmate.name}"

    def report_behavior(self, inmate, behavior_record):
        inmate.add_behavior_record(behavior_record)
        return f"Behavior recorded for inmate {inmate.name}: {behavior_record.description}"

# WorkAssignment Class
class WorkAssignment:
    def __init__(self, assignment_name):
        self.assignment_name = assignment_name
        self.assigned_inmates = []  # List of Inmate objects

    def assign_inmate(self, inmate):
        self.assigned_inmates.append(inmate)
        return f"Inmate {inmate.name} assigned to work {self.assignment_name}"

    def remove_inmate(self, inmate):
        if inmate in self.assigned_inmates:
            self.assigned_inmates.remove(inmate)
            return f"Inmate {inmate.name} removed from work assignment {self.assignment_name}"
        else:
            return f"Inmate {inmate.name} is not assigned to {self.assignment_name}"

    def track_progress(self, inmate):
        return f"Tracking work progress for inmate {inmate.name} in {self.assignment_name}"


Create and Populate the Database

In [10]:
# Create SQLite Database
engine = create_engine('sqlite:///prison_system.db')
Base.metadata.create_all(engine)

# Create a session for interacting with the database
Session = sessionmaker(bind=engine)
session = Session()

# Helper functions to generate random data
def random_name():
    first_names = ["John", "Jane", "Alex", "Sam", "Chris", "Taylor", "Jordan", "Morgan", "Robin", "Skyler"]
    last_names = ["Smith", "Johnson", "Brown", "Williams", "Jones", "Garcia", "Martinez", "Robinson", "Clark", "Lewis"]
    return f"{random.choice(first_names)} {random.choice(last_names)}"

def random_crime():
    crimes = ["Theft", "Assault", "Fraud", "Drug Possession", "Robbery", "Arson", "Murder", "Tax Evasion"]
    return random.choice(crimes)

def random_behavior():
    behaviors = [("Good", "Helped with prison maintenance"),
                 ("Bad", "Started a fight"),
                 ("Good", "Volunteered for work assignment"),
                 ("Bad", "Attempted escape")]
    return random.choice(behaviors)

# Populate the database with random data
def populate_database(session):
    # Generate 5 Cells
    cells = []
    for i in range(5):
        cell = Cell(cell_number=100 + i, capacity=10)
        cells.append(cell)
        session.add(cell)

    # Generate 50 Inmates
    for i in range(50):
        name = random_name()
        age = random.randint(20, 60)
        crime = random_crime()
        sentence_duration = random.randint(1, 25)
        time_served = round(random.uniform(0, sentence_duration), 1)

        # Randomly assign an inmate to a cell
        cell = random.choice(cells)

        # Create a MedicalRecord for the inmate
        medical_condition = random.choice(["Healthy", "Diabetes", "Asthma", "Heart Disease", "Hypertension"])
        treatment_plan = f"Treatment for {medical_condition}"
        medical_record = MedicalRecord(medical_condition=medical_condition, treatment_plan=treatment_plan)

        # Create a Parole status for the inmate
        parole_date = datetime.date.today() + datetime.timedelta(days=random.randint(100, 1000))
        parole_eligibility = random.choice([True, False])
        parole_status = Parole(parole_date=parole_date, eligibility_status=parole_eligibility)

        # Create the Inmate object
        inmate = Inmate(
            name=name,
            age=age,
            crime=crime,
            sentence_duration=sentence_duration,
            time_served=time_served,
            cell=cell,
            medical_records=medical_record,
            parole_status=parole_status
        )

        # Add random behavior records for the inmate
        for _ in range(random.randint(1, 5)):  # Randomly add 1-5 behavior records
            behavior_type, description = random_behavior()
            behavior_record = BehaviorRecord(
                date=datetime.date.today() - datetime.timedelta(days=random.randint(1, 365)),
                description=description,
                behavior_type=behavior_type
            )
            inmate.behavior_records.append(behavior_record)

        # Add the inmate and related records to the session
        session.add(inmate)

    # Commit the session to store all entries in the database
    session.commit()

# Populate the database with random data
populate_database(session)


Export to CSV and Download

In [11]:
# Export inmate data to CSV
def export_inmates_to_csv(session, file_path):
    # Query all inmates from the database
    inmates_data = session.query(Inmate).all()

    # Prepare data for CSV export
    inmate_rows = []

    for inmate in inmates_data:
        # Create a dictionary for each inmate's information
        inmate_dict = {
            "Name": inmate.name,
            "Age": inmate.age,
            "Crime": inmate.crime,
            "Sentence Duration": inmate.sentence_duration,
            "Time Served": inmate.time_served,
            "Cell Number": inmate.cell.cell_number if inmate.cell else None,
            "Medical Condition": inmate.medical_records.medical_condition if inmate.medical_records else None,
            "Parole Eligibility": inmate.parole_status.eligibility_status if inmate.parole_status else None,
            "Parole Date": inmate.parole_status.parole_date if inmate.parole_status else None
        }

        # Add behavior records to the dictionary
        for idx, behavior in enumerate(inmate.behavior_records, start=1):
            inmate_dict[f"Behavior {idx}"] = f"{behavior.behavior_type} - {behavior.description} on {behavior.date}"

        inmate_rows.append(inmate_dict)

    # Convert the list of dictionaries to a pandas DataFrame
    df_inmates = pd.DataFrame(inmate_rows)

    # Save the DataFrame to a CSV file
    df_inmates.to_csv(file_path, index=False)
    print(f"Data exported successfully to {file_path}")

# Export and download the CSV
def export_and_download_csv(session, file_path='inmates_data.csv'):
    export_inmates_to_csv(session, file_path)
    files.download(file_path)

# Export and download CSV
export_and_download_csv(session)


Data exported successfully to inmates_data.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>